##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Leitura de banco de dados PostgreSQL pelo TensorFlow IO

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/postgresql"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/postgresql.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/postgresql.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/io/tutorials/postgresql.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Este tutorial mostra como criar um `tf.data.Dataset` a partir de um servidor de banco de dados PostgreSQL de forma que o `Dataset` criado possa ser passado ao `tf.keras` para treinamento e inferência.

Bancos de dado SQL são uma fonte de dados importante para cientistas de dados. Por ser um dos bancos de dados SQL de código aberto mais populares, o [PostgreSQL](https://www.postgresql.org) é amplamente usado em empresas para armazenar dados críticos e transacionais. Criar um `Dataset` a partir de um servidor de banco de dados PostgreSQL diretamente e passar o `Dataset` ao `tf.keras` para treinamento e inferência pode simplificar bastante o pipeline da dados e ajudar os cientistas de dados a se concentrarem em criar modelos de aprendizado de máquina.

## Configuração e uso

### Instale os pacotes tensorflow-io necessários e reinicie o runtime

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

!pip install tensorflow-io

### Instale e configure o PostgreSQL (opcional)

**Atenção: este notebook foi criado para execução somente no Google Colab**. *Ele instala pacotes no sistema e requer acesso sudo. Se você quiser executar em um notebook Jupyter local, prossiga com cautela.*

Para demonstrar o uso no Google Colab, você instalará o servidor PostgreSQL. Também serão necessários a senha e um banco de dados vazio.

Se você não estiver executando este notebook no Google Colab ou se preferir usar um banco de dados existente, pule a configuração abaixo e avance para a próxima seção.

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS tfio_demo;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE tfio_demo;'

### Configure as variáveis de ambiente necessárias

As variáveis de ambiente abaixo são baseadas na configuração do PostgreSQL da última seção. Se você tiver uma configuração diferente ou se estiver utilizando um banco de dados existente, altere as variáveis conforme necessário:


In [ ]:
%env TFIO_DEMO_DATABASE_NAME=tfio_demo
%env TFIO_DEMO_DATABASE_HOST=localhost
%env TFIO_DEMO_DATABASE_PORT=5432
%env TFIO_DEMO_DATABASE_USER=postgres
%env TFIO_DEMO_DATABASE_PASS=postgres

### Prepare os dados no servidor PostgreSQL

Para fins de demonstração deste tutorial, você vai criar um banco de dados e preenchê-lo com alguns dados. Os dados usados no tutorial são do [dataset Air Quality](https://archive.ics.uci.edu/ml/datasets/Air+Quality) (qualidade do ar), disponível no [Repositório de aprendizado de máquina da UCI](http://archive.ics.uci.edu/ml).

Veja abaixo uma prévia de um subconjunto do dataset Air Quality:

Data | Horário | CO(GT) | PT08.S1(CO) | NMHC(GT) | C6H6(GT) | PT08.S2(NMHC) | NOx(GT) | PT08.S3(NOx) | NO2(GT) | PT08.S4(NO2) | PT08.S5(O3) | T | UR | UA
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---
10/03/2004 | 18.00.00 | 2,6 | 1360 | 150 | 11,9 | 1046 | 166 | 1056 | 113 | 1692 | 1268 | 13,6 | 48,9 | 0,7578
10/03/2004 | 19.00.00 | 2 | 1292 | 112 | 9,4 | 955 | 103 | 1174 | 92 | 1559 | 972 | 13,3 | 47,7 | 0,7255
10/03/2004 | 20.00.00 | 2,2 | 1402 | 88 | 9,0 | 939 | 131 | 1140 | 114 | 1555 | 1074 | 11,9 | 54,0 | 0,7502
10/03/2004 | 21.00.00 | 2,2 | 1376 | 80 | 9,2 | 948 | 172 | 1092 | 122 | 1584 | 1203 | 11,0 | 60,0 | 0,7867
10/03/2004 | 22.00.00 | 1,6 | 1272 | 51 | 6,5 | 836 | 131 | 1205 | 116 | 1490 | 1110 | 11,2 | 59,6 | 0,7888

Confira mais informações sobre o dataset Air Quality e o Repositório de aprendizado de máquina da UCI na seção [Referências](#references).

Para ajudar a simplificar a preparação dos dados, preparamos uma versão em SQL do dataset Air Quality, disponível como [AirQualityUCI.sql](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/postgresql/AirQualityUCI.sql).

A declaração para criar a tabela é:

```
CREATE TABLE AirQualityUCI (
  Date DATE,
  Time TIME,
  CO REAL,
  PT08S1 INT,
  NMHC REAL,
  C6H6 REAL,
  PT08S2 INT,
  NOx REAL,
  PT08S3 INT,
  NO2 REAL,
  PT08S4 INT,
  PT08S5 INT,
  T REAL,
  RH REAL,
  AH REAL
);
```

Veja os comandos completos para criar a tabela no banco de dados e preencher com os dados:

In [ ]:
!curl -s -OL https://github.com/tensorflow/io/raw/master/docs/tutorials/postgresql/AirQualityUCI.sql

!PGPASSWORD=$TFIO_DEMO_DATABASE_PASS psql -q -h $TFIO_DEMO_DATABASE_HOST -p $TFIO_DEMO_DATABASE_PORT -U $TFIO_DEMO_DATABASE_USER -d $TFIO_DEMO_DATABASE_NAME -f AirQualityUCI.sql

### Crie um dataset a partir do servidor PostgreSQL e utilize-o no TensorFlow

Criar um dataset a partir do servidor PostgreSQL é fácil, basta chamar `tfio.experimental.IODataset.from_sql` com os argumentos `query` (consulta) e `endpoint`. `query` é a consulta SQL para selecionar colunas nas tabelas, e `endpoint` é o endereço e nome do banco de dados:

In [ ]:
import os
import tensorflow_io as tfio

endpoint="postgresql://{}:{}@{}?port={}&dbname={}".format(
    os.environ['TFIO_DEMO_DATABASE_USER'],
    os.environ['TFIO_DEMO_DATABASE_PASS'],
    os.environ['TFIO_DEMO_DATABASE_HOST'],
    os.environ['TFIO_DEMO_DATABASE_PORT'],
    os.environ['TFIO_DEMO_DATABASE_NAME'],
)

dataset = tfio.experimental.IODataset.from_sql(
    query="SELECT co, pt08s1 FROM AirQualityUCI;",
    endpoint=endpoint)

print(dataset.element_spec)

Como podemos ver na saída de `dataset.element_spec` acima, o elemento do `Dataset` criado é um objeto dict (dicionário) do Python, com nomes de colunas da tabela do banco de dados usadas como chaves. É muito simples aplicar outras operações. Por exemplo, você pode selecionar tanto o campo `nox` quanto o campo `no2` do `Dataset` e calcular a diferença:

In [ ]:
dataset = tfio.experimental.IODataset.from_sql(
    query="SELECT nox, no2 FROM AirQualityUCI;",
    endpoint=endpoint)

dataset = dataset.map(lambda e: (e['nox'] - e['no2']))

# check only the first 20 record
dataset = dataset.take(20)

print("NOx - NO2:")
for difference in dataset:
  print(difference.numpy())

Agora, o `Dataset` criado está pronto para ser passado ao `tf.keras` diretamente para treinamento ou inferência.

## Referências

- Dua, D. e Graff, C. (2019). Repositório de aprendizado de máquina da UCI [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.
- S. De Vito, E. Massera, M. Piga, L. Martinotto, G. Di Francia, On field calibration of an electronic nose for benzene estimation in an urban pollution monitoring scenario, Sensors and Actuators B: Chemical, volume 129, edição 2, 22 de fevereiro de 2008, páginas 750-757, ISSN 0925-4005 (Calibração em campo de um nariz eletrônico para estimativa de benzeno em um cenário de monitoramento de poluição urbana)